# BankLab: JPM vs MS Analysis

This notebook demonstrates basic analysis using BankLab's processed data.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Load processed data
DATA_DIR = Path('../data/processed')

## 1. Price Data Analysis

In [ ]:
# Load prices
prices = pd.read_parquet(DATA_DIR / 'prices_daily.parquet')
prices['date'] = pd.to_datetime(prices['date'])

print(f"Loaded {len(prices)} price records")
print(f"Date range: {prices['date'].min()} to {prices['date'].max()}")
prices.head()

In [ ]:
# Plot cumulative returns
fig, ax = plt.subplots(figsize=(12, 6))

for ticker in ['JPM', 'MS']:
    ticker_data = prices[prices['ticker'] == ticker].copy()
    ticker_data['cum_ret'] = (1 + ticker_data['ret'].fillna(0)).cumprod()
    ax.plot(ticker_data['date'], ticker_data['cum_ret'], label=ticker)

ax.set_title('Cumulative Returns: JPM vs MS')
ax.set_xlabel('Date')
ax.set_ylabel('Cumulative Return')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 2. Factor Exposures

In [ ]:
# Load factors
factors = pd.read_parquet(DATA_DIR / 'factors_daily.parquet')
factors['date'] = pd.to_datetime(factors['date'])

print(f"Loaded {len(factors)} factor observations")
factors.head()

In [ ]:
# Merge prices with factors for regression
import statsmodels.api as sm

def run_ff5_regression(ticker_returns, factors_df):
    """Run Fama-French 5-factor regression."""
    merged = ticker_returns.merge(factors_df, on='date', how='inner')
    merged = merged.dropna()
    
    # Excess returns
    merged['excess_ret'] = merged['ret'] - merged['rf']
    
    # Regression
    X = merged[['mktrf', 'smb', 'hml', 'rmw', 'cma']]
    X = sm.add_constant(X)
    y = merged['excess_ret']
    
    model = sm.OLS(y, X).fit()
    return model

# Run for each ticker
for ticker in ['JPM', 'MS']:
    ticker_data = prices[prices['ticker'] == ticker][['date', 'ret']].copy()
    model = run_ff5_regression(ticker_data, factors)
    print(f"\n{ticker} Factor Exposures:")
    print(model.summary().tables[1])

## 3. Fundamentals Overview

In [ ]:
# Load fundamentals
fundamentals = pd.read_parquet(DATA_DIR / 'fundamentals_raw_facts.parquet')
fundamentals['date'] = pd.to_datetime(fundamentals['date'])

print(f"Loaded {len(fundamentals)} fundamental facts")
print(f"Unique tags: {fundamentals['tag'].nunique()}")
fundamentals.head()

In [ ]:
# Example: Extract key metrics
key_tags = [
    'us-gaap:Assets',
    'us-gaap:Revenues',
    'us-gaap:NetIncomeLoss',
    'us-gaap:StockholdersEquity',
]

for ticker in ['JPM', 'MS']:
    print(f"\n{ticker} Key Metrics (most recent):")
    ticker_data = fundamentals[
        (fundamentals['ticker'] == ticker) & 
        (fundamentals['tag'].isin(key_tags)) &
        (fundamentals['form'].isin(['10-K', '10-Q']))
    ]
    
    for tag in key_tags:
        tag_data = ticker_data[ticker_data['tag'] == tag]
        if len(tag_data) > 0:
            latest = tag_data.sort_values('date').iloc[-1]
            print(f"  {tag.split(':')[1]}: ${latest['value']:,.0f} ({latest['date'].date()})")